In [1]:
import setup

setup.init_django()

In [2]:
from openai import OpenAI
from blog.models import BlogPost
from decouple import config
EMEDDING_MODEL=config("EMEDDING_MODEL", default="text-embedding-3-small")
RECREATE_DATA=True

In [3]:
querry_set=BlogPost.objects.all()
querry_set

<QuerySet [<BlogPost: BlogPost object (69)>, <BlogPost: BlogPost object (70)>, <BlogPost: BlogPost object (71)>, <BlogPost: BlogPost object (72)>]>

In [4]:
docs = [
    "The dog jumped over the cat", 
    "The cat jumped over the dog",
    "It is very warm today",
    "The cat is yellow and the dog is red",
]

In [5]:
new_data = []
for i, x in enumerate(docs):
  new_data.append(BlogPost.objects.create(title=f"Blog Post {i+1}", can_delete=True))
new_data


if RECREATE_DATA:
    qs = BlogPost.objects.filter(can_delete=True)
    qs.delete()
    BlogPost.objects.bulk_create(new_data)

In [6]:
qs = BlogPost.objects.filter(can_delete=True)
qs.count()

4

In [7]:
client = OpenAI(
    api_key=config("OPENAI_API_KEY")
)

In [8]:
def get_embedding(text, model=EMEDDING_MODEL):
    text = text.replace("\n", " ").strip()
    return client.embeddings.create(input=[text], model=model).data[0].embedding

In [9]:
EMEDDING_MODEL

'text-embedding-3-small'

In [11]:
for obj in qs:
    if obj.embedding is None:
        obj.embedding = get_embedding(obj.get_embedding_text_raw())
        obj.save()

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}